In [1]:
"""Notes/instructions on how to run experiments.

"""

'Notes/instructions on how to run experiments.\n\n'

In [2]:
from nna.fileUtils import list_files,save_to_csv

from pathlib import Path
from nna.pre_process_func import read_queue

import pandas as pd

In [3]:
# originally from ../src/notes.md

* start with screen
* get GPU Memory so I can use it later
* one file file at a time (~2 gb)
  * I will divide files to 1 hour slices
  * I will use 25 cpus so 25 processes
  * embeddings file name will be filename_embeddings.



1) create a text file with paths to input audio files (one full path per line)
 
* `find /tank/data/nna/real/ -iname "*flac" > ExperimentRunVXXX.txt`
* or following code, list_files function

v2) since now we have new files in the old folders, we need to find which files has results which files do not,
we could let the code ignore existing results however, ffmpeg divides files into couple of hours of segments without checking results.



In [4]:
datafiles_path="/home/enis/projects/nna/data/"


In [293]:
search_path="/tank/data/nna/real/"
ignore_folders=["/tank/data/nna/real/ivvavik/","/tank/data/nna/real/stinchcomb/dups/","/tank/data/nna/real/stinchcomb/excerpts/"]

files_path_list=list_files(search_path,ignore_folders)
files_path_list=list(set(files_path_list))

datafiles_path="/home/enis/projects/nna/data/"
input_filename = "ExperimentRunV5.txt"

thepath = datafiles_path + input_filename
with open(thepath,"w") as myfile:
    myfile.writelines("\n".join(files_path_list))
    

2) update input, output directory, input list,  in /nna/src/params.py

3-1) sync files 

`rsync -av --recursive --update ./ enis@crescent:/home/enis/projects/nna/ `

3-2) run codes
```bash
python3 ./src/scripts/pre_process.py &>> job_logs/logs.txt; python /home/enis/projects/nna/src/nna/slack_message.py -t "cpu job ended" -f job_logs/logs.txt &

python3 ./src/scripts/watch_VGGish.py &>> job_logs/logs_gpu.txt; python /home/enis/projects/nna/src/nna/slack_message.py -t "gpu job stopped" & 
```
4) to re-run, update the code and remove temporary flac files

`rsync -av --recursive --update /Users/berk/Documents/workspace/speech_audio_understanding/src/ enis@crescent:/home/enis/projects/nna/`

`find /scratch/enis/data/nna/real/ -iname "*flac"  -delete`

`find /scratch/enis/data/nna/real/ -path "*/*_segments*/*" -delete`

`find /scratch/enis/data/nna/real/ -name "*_segments" -type d -delete`

also remove `job_logs/pre_processing_queue.csv` if jobs left unfinished

and STOP all processes

5) tracking progress 
```bash
cat job_logs/pre_processing_queue.csv | wc -l; cat job_logs/pre_processed_queue.csv | wc -l; cat job_logs/VGGISH_processing_queue.csv | wc -l; cat job_logs/vggish_embeddings_queue.csv | wc -l;
```
6) backup
```bash
tar cf - /scratch/enis/data/nna/backup/NUI_DATA/ -P | pv -s $(du -sb /scratch/enis/data/nna/backup/NUI_DATA/ | awk '{print $1}') | gzip > embeddings_backup.tar.gz
```


In [9]:
q1=read_queue("../src/job_logs/pre_processing_queue.csv")
q2=read_queue("../src/job_logs/pre_processed_queue.csv")
q3=read_queue("../src/job_logs/VGGISH_processing_queue.csv")
q4=read_queue("../src/job_logs/vggish_embeddings_queue.csv")
q5=read_queue("../src/job_logs/Audioset_processing_queue.csv")
q6=read_queue("../src/job_logs/Audioset_output_queue.csv")

CABLE=read_queue("../src/job_logs/_CABLE_output_queue.csv")



In [142]:
q1set=set(["/".join(i.split("/")[-1:])[:-5] for i in q1])
q2set=set(["/".join(i.split("/")[-2:-1])[:-13] for i in q2])


In [144]:
print(q1set.difference(q2set))
print(q2set.difference(q1set))

{'SINP-05_20190709_090000', 'S4A10252_20190529_171602', 'S4A10259_20190514_183000'}


25.64% is NOT finished


In [8]:
# files without results (subDirectoryAddons)
# this one goes through files in the properties_df and checks if there are corresponding files with given file name add ons.

from nna.fileUtils import standard_path_style
import pandas as pd

# file_properties_df_ST=pd.read_pickle("../data/stinchcomb_dataV1.pkl")
file_properties_df_ST = []
# file_properties_df = pd.read_pickle("../data/realdata_v2No_stinchcomb.pkl")
# file_properties_df = pd.read_pickle("../data/prudhoeAndAnwr4photoExp_dataV1.pkl")
# file_properties_df = pd.read_pickle("/home/enis/projects/nna/data/prudhoeAndAnwr4photoExp_dataV1.pkl")
file_properties_df = pd.read_pickle('/scratch/enis/data/nna/database/allFields_dataV5.pkl')

# subDirectoryAddon="vgg"
# subDirectoryAddon="_CABLE"
# subDirectoryAddon="_CABLE"
# subDirectoryAddons=["CABLE","RUNNINGWATER","INSECT", "RAIN", "WATERBIRD", "WIND", "SONGBIRD", "AIRCRAFT"]
subDirectoryAddons=["3rk9ayjc-V1-0-0-0"]
notDoneFilesDict={}
for subDirectoryAddon in subDirectoryAddons:
    print(subDirectoryAddon)
    notDoneFiles=[]
    
    for afile,row in file_properties_df.iterrows():
        checkFile = standard_path_style("/scratch/enis/data/nna/real/",row,sub_directory_addon=subDirectoryAddon,file_name_addon="")
        if not checkFile.exists():
            notDoneFiles.append(afile)
    
    if file_properties_df_ST != []:
        for afile,row in file_properties_df_ST.iterrows():
            checkFile = standard_path_style("/scratch/enis/data/nna/real/",row,sub_directory_addon=subDirectoryAddon,file_name_addon="")
            if not checkFile.exists():
        #         print(afile)
    #             print(checkFile)
                notDoneFiles.append(afile)

    notDoneFiles.sort()
    notDoneFiles=[str(i) for i in notDoneFiles]
    notDoneFilesDict[subDirectoryAddon]=notDoneFiles[:]


# input_filename = "ExperimentRunV6.txt"

# thepath=datafiles_path+input_filename
# with open(thepath,"w") as myfile:
#     myfile.writelines("\n".join(notDoneFiles))
total_input_file_count = len(file_properties_df)
count_not_done_files = len(notDoneFiles)
print(f'{(count_not_done_files/total_input_file_count)*100:.2f}% is NOT finished')



3rk9ayjc-V1-0-0-0
4.13% is NOT finished


In [26]:
print('asd')

asd


In [ ]:
"77.62% "

In [124]:
# len(notDoneFiles) 13729

13729

In [99]:
# len(noEmbeddings)

4937

In [37]:
thepath

'/home/enis/projects/nna/data/inferenceInputs/allModels6tagRunV1_20191102.txt'

In [36]:
input_filename = "allModels6tagRunV1_20191102.txt"

thepath=datafiles_path+"inferenceInputs/"+input_filename
# thepath
with open(thepath,"w") as myfile:
    myfile.writelines("\n".join([str(i) for i in yesEmbeddings]))

In [10]:
# from missing results, find missing embeddings
# in the next part, we find missing embeddings by searching folders, here with by iterating file_properties_df
noEmbeddings=[]
yesEmbeddings=[]
for f in notDoneFiles:
    row = file_properties_df.loc[Path(f)]
    checkFile= standard_path_style("/scratch/enis/data/nna/real/",row,subDirectoryAddon="_vgg",fileNameAddon="_rawembeddings000.npy")
    if not checkFile.exists():
#         print(checkFile)
        noEmbeddings.append(checkFile)
    else:
        yesEmbeddings.append(checkFile)
    

In [11]:
len(noEmbeddings)

11

In [100]:
# check if any of the missing files were in the previous experiment
inputFilesOld="/scratch/enis/data/nna/real/input_2020-10-12_17:11:26.txt"
with open(inputFilesOld) as ff:
    ffL=ff.readlines()
    ffL = [i.strip().split("\t")[0] for i in ffL]

noEmbeddings2=[]
yesEmbeddings2=[]
for f in notDoneFiles:
    row = file_properties_df.loc[Path(f)]
    checkFile= standard_path_style("/scratch/enis/data/nna/real/",row,subDirectoryAddon="_vgg",fileNameAddon="_rawembeddings000.npy")
    if f not in ffL:
#         print(checkFile)
        noEmbeddings2.append(checkFile)
    else:
        yesEmbeddings2.append(checkFile)

setnoEmbeddings = set(noEmbeddings)
setnoEmbeddings2 = set(noEmbeddings2)
print("Following ones are in the input but do not have embeddings")
setnoEmbeddings.difference(setnoEmbeddings2)

In [108]:
len(setnoEmbeddings2)

4934

In [136]:
t=[]
for i in yesEmbeddings[0:10]:
    t.append(str(i))

In [137]:
for i in yesEmbeddings[-20:-10]:
    t.append(str(i))

### FIND missing prediction results

In [123]:
import glob
inputCSV=[]
# for i in ["ivvavik","anwr",  "prudhoe" , "stinchcomb"]:
for i in ["anwr",  "prudhoe" ]:
    x=glob.glob( f"/scratch/enis/data/nna/real/{i}/**/*_rawembeddings*.npy",recursive=True)
    print(i,len(x))
    inputCSV.extend(x)


# save_to_csv("../src/job_logs/inferenceRun3.csv",[[str(afile)] for afile in inputCSV])


anwr 12388
prudhoe 13148


In [142]:
save_to_csv("/home/enis/projects/nna/job_logs/inferenceRunVtest.csv",[[str(afile)] for afile in t])


In [ ]:
# _preprocessed.npy
search_path="/tank/data/nna/real/"
ignore_folders=["/tank/data/nna/real/ivvavik/","/tank/data/nna/real/stinchcomb/dups/","/tank/data/nna/real/stinchcomb/excerpts/"]

files_path_list=list_files(search_path,ignore_folders)
files_path_list=list(set(files_path_list))

datafiles_path="/home/enis/projects/nna/data/"
input_filename = "ExperimentRunV5.txt"

thepath=datafiles_path+input_filename
with open(thepath,"w") as myfile:
    myfile.writelines("\n".join(files_path_list))
    

### Run predictions

In [132]:
#TODO all these are running on raw embeddings, makes sure thats checked before running on the input

In [ ]:

inputCsvFile="./job_logs/inferenceRun3.csv"

inputCsvFile="/home/enis/projects/nna/data/inferenceInputs/allModels6tagRunV1_20191102.txt"
outputLogs="job_logs/MLlogs.txt"
i=0
python3 ./src/scripts/watchMLInference.py --inputCsv $inputCsvFile --modelPath "./src/nna/assets/sklearnModels/Cable_Gaussian Process_Raw_Concat_2020-03-02--14-06.joblib"  --modelName "_CABLE" &>> $outputLogs & 
pids[i]=$! && ((i=i+1));
python3 ./src/scripts/watchMLInference.py --inputCsv $inputCsvFile --modelPath "./src/nna/assets/sklearnModels/Running Water_Neural Net_Raw_Concat_2020-03-02--14-06.joblib"  --modelName "_RUNNINGWATER" &>> $outputLogs &
pids[i]=$! && ((i=i+1));
python3 ./src/scripts/watchMLInference.py --inputCsv $inputCsvFile --modelPath "./src/nna/assets/sklearnModels/Insect_Linear SVM_Raw_Concat_2020-03-02--14-06.joblib"  --modelName "_INSECT" &>> $outputLogs & 
pids[i]=$! && ((i=i+1));
python3 ./src/scripts/watchMLInference.py --inputCsv $inputCsvFile --modelPath "./src/nna/assets/sklearnModels/Rain_Gaussian Process_Raw_Concat_2020-03-02--14-06.joblib"  --modelName "_RAIN" &>> $outputLogs &
pids[i]=$! && ((i=i+1));
python3 ./src/scripts/watchMLInference.py --inputCsv $inputCsvFile --modelPath "./src/nna/assets/sklearnModels/Water Bird_RBF SVM_Raw_many2one_2020-03-02--14-06.joblib"  --modelName "_WATERBIRD" &>> $outputLogs &
pids[i]=$! && ((i=i+1));
python3 ./src/scripts/watchMLInference.py --inputCsv $inputCsvFile --modelPath "./src/nna/assets/sklearnModels/Wind_Neural Net_Raw_many2one_2020-03-02--14-06.joblib"  --modelName "_WIND" &>> $outputLogs &
pids[i]=$! && ((i=i+1));
python3 ./src/scripts/watchMLInference.py --inputCsv $inputCsvFile --modelPath "./src/nna/assets/sklearnModels/Songbird_Neural Net_Raw_many2one_2020-03-02--14-06.joblib"  --modelName "_SONGBIRD" &>> $outputLogs &
pids[i]=$! && ((i=i+1));
python3 ./src/scripts/watchMLInference.py --inputCsv $inputCsvFile --modelPath "./src/nna/assets/sklearnModels/Aircraft_AdaBoost_Raw_Average_2020-03-02--14-06.joblib"  --modelName "_AIRCRAFT" &>> $outputLogs &
pids[i]=$! && ((i=i+1));
for pid in ${pids[*]}; do
    wait $pid
done;
python3 slack_message.py -t "MLInference job ended" -f $outputLogs &


    AIRCRAFT,SONGBIRD,WATERBIRD,WIND

In [39]:
# set(inputCSV)
import time
logsPath="../src/job_logs/"
inputCSVset=set(inputCSV)
for subDirectoryAddon in subDirectoryAddons:
    csvFile=logsPath+subDirectoryAddon+"_output_queue.csv"
    print(subDirectoryAddon)
    csvFileSet=set(read_queue(csvFile))
    print(len(inputCSVset.difference(csvFileSet)))
#     for i in inputCSVset.difference(csvFileSet):
#         print(i)


NameError: name 'inputCSV' is not defined